In [16]:
import sys, importlib
sys.path.append(r"C:\Users\DARY\Desktop\Dary\Prueba-BigData\Proyecto-hive\scripts")

import parametrossh
importlib.reload(parametrossh)

from parametrossh import get_params, init_spark, get_sqlserver_connection


In [30]:
# 1. Iniciar sesión de Spark
spark = init_spark(1, "CargaPedidos")
print("✅ SparkSession inicializada")


✅ SparkSession inicializada


In [ ]:
# 2. Conexión SQL Server
jdbc_url, properties = get_sqlserver_connection(1)

# 3. Leer tabla Pedidos
df_pedidos = spark.read.jdbc(
    url=jdbc_url,
    table="dbo.Pedidos",
    properties=properties
)
print("✅ Pedidos desde SQL Server")
df_pedidos.show()

✅ Pedidos desde SQL Server


[Stage 0:>                                                          (0 + 1) / 1]

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
+--------+---------+------+-----------+--------------------+----------+
only showing top 5 rows



In [19]:
df=spark.read.parquet('hdfs://namenode:8020/bronze/incremental_pedidos')
df.show(truncate=False)

[Stage 2:>                                                          (0 + 1) / 1]

+-----------------------+-------+
|last_watermark         |tabla  |
+-----------------------+-------+
|2025-09-30 15:41:09.443|pedidos|
+-----------------------+-------+



In [20]:
df1 = df.filter(df.tabla == "pedidos") \
                       .select("last_watermark") \
                       .collect()[0][0]

print("🕒 Última marca de agua usada:", df1)

[Stage 3:>                                                          (0 + 1) / 1]

🕒 Última marca de agua usada: 2025-09-30 15:41:09.443000


In [21]:
# 1. Conexión SQL parametrizada
jdbc_url, properties = get_sqlserver_connection(1)

# 2. Construir query incremental
query_pedidos_incr = f"""
(
   SELECT *
   FROM dbo.Pedidos
   WHERE CreateTime > '{df1}'
) AS pedidos_incr
"""


In [25]:
# 3. Leer desde SQL Server
df_incremental = spark.read.jdbc(
    url=jdbc_url,
    table=query_pedidos_incr,
    properties=properties
)

# 4. Validar
if df_incremental.rdd.isEmpty():
    print("⚠️ No hay registros nuevos después de", df1)
else:
    print("✅ Nuevos registros encontrados:", df_incremental.count())
    df_incremental.show(5)

✅ Nuevos registros encontrados: 1
+--------+---------+-------+-----------+--------------------+----------+
|PedidoID|ClienteID|  Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+-------+-----------+--------------------+----------+
|      23|       16|5000.50| 2025-09-30|2025-09-30 15:57:...|      null|
+--------+---------+-------+-----------+--------------------+----------+



In [27]:
df_incremental.write.mode("append").parquet("hdfs://namenode:8020/bronze/f_pedidos")
print("✅ Datos incrementales guardados en Bronze/pedidos")

✅ Datos incrementales guardados en Bronze/pedidos


In [32]:
df_pedidos = spark.read.parquet("hdfs://namenode:8020/bronze/df_pedidos")
df_pedidos.show()


[Stage 3:>                                                          (0 + 1) / 1]

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
|       6|        1| 80.00| 2025-09-06|2025-09-14 19:36:...|      null|
|       7|        2| 60.00| 2025-09-07|2025-09-14 19:36:...|      null|
|       8|        3| 90.00| 2025-09-08|2025-09-14 19:36:...|      null|
|       9|        4|110.00| 2025-09-09|2025-09-14 19:36:...|      null|
|      10|        5| 75.00| 2025-09-10|2025-09-14 19:36:...|      null|
|      11|        7| 95.00| 2025-09-11|2025-09-14 19:36:...|    

In [33]:
from pyspark.sql.functions import max as spark_max, lit

marca_df = df_pedidos.agg(spark_max("CreateTime").alias("last_watermark")) \
                  .withColumn("tabla", lit("pedidos"))

In [34]:
marca_df.show(truncate=False)

[Stage 4:>                                                          (0 + 1) / 1]

+-----------------------+-------+
|last_watermark         |tabla  |
+-----------------------+-------+
|2025-09-30 15:57:00.061|pedidos|
+-----------------------+-------+



In [36]:
marca_df.write.mode('overwrite').parquet('hdfs://namenode:8020/bronze/incremental_pedidos')

In [37]:
df=spark.read.parquet('hdfs://namenode:8020/bronze/incremental_pedidos')
df.show(truncate=False)

+-----------------------+-------+
|last_watermark         |tabla  |
+-----------------------+-------+
|2025-09-30 15:57:00.061|pedidos|
+-----------------------+-------+



In [38]:
spark.stop()